In [3]:
import tensorflow as tf

import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [1]:
# examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
#                                as_supervised=True)
                               
# train_examples, val_examples = examples['train'], examples['validation']